# Prerequisites

In [1]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install tqdm
!pip install torchvision
!pip install seaborn
!pip install sklearn
!pip install numpy
!pip install opencv-python


# ## commands to install the env
!git clone https://github.com/MultiAgentLearning/playground ./pommer_setup
!pip install -U ./pommer_setup
!rm -rf ./pommer_setup

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1MB 1.3MB/s eta 0:14:50tcmalloc: large alloc 1147494400 bytes == 0x55bf050a0000 @  0x7ff9ab500615 0x55becc883cdc 0x55becc96352a 0x55becc886afd 0x55becc977fed 0x55becc8fa988 0x55becc8f54ae 0x55becc8883ea 0x55becc8fa7f0 0x55becc8f54ae 0x55becc8883ea 0x55becc8f732a 0x55becc978e36 0x55becc8f6853 0x55becc978e36 0x55becc8f6853 0x55becc978e36 0x55becc8f6853 0x55becc978e36 0x55becc9fb3e1 0x55becc95b6a9 0x55becc8c6cc4 0x55becc887559 0x55becc8fb4f8 0x55becc88830a 0x55becc8f63b5 0x55becc8f57ad 0x55becc8883ea 0x55becc8f63b5 0x55becc88830a 0x55becc8f63b5
     |█████████████████               | 1055.7MB 1.2MB/s eta 0:12:27tcmalloc: large alloc 1434370048 bytes == 0x55bf496f6000 @  0x7ff9ab500615 0x55becc883cdc 0x55becc96352a 0x55becc886afd 0x55becc977fed 0x55becc8fa988 0x55becc8f54ae 0x55becc8883ea 0x55becc8fa7f0 0x55becc8f54ae 0x55becc8883ea 0x55becc8f732a 0x55becc978e36 0x55becc8f6853 0

In [2]:
!pip install onnx
!pip install onnx2pytorch
import onnx
from onnx2pytorch import ConvertModel
import random

     |████████████████████████████████| 12.2MB 11.0MB/s 


In [3]:
!git clone https://github.com/RLCommunity/graphic_pomme_env ./graphic_pomme_env
!pip install -U ./graphic_pomme_env
!rm -rf ./graphic_pomme_env

Cloning into './graphic_pomme_env'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 2), reused 14 (delta 1), pack-reused 0
Unpacking objects: 100% (15/15), done.
Processing ./graphic_pomme_env
  Created wheel for graphic-pomme-env: filename=graphic_pomme_env-0.1.0-cp37-none-any.whl size=8057 sha256=c42119e7731c9c894d7a9ffb5444d03d051568bc4d3849cc9f0b69bf2621d721
  Stored in directory: /tmp/pip-ephem-wheel-cache-k9f8ae93/wheels/eb/6e/62/9a49cdf37b0a50b0ab19e5a10e86acdd7d2c6069cb505148df
Successfully built graphic-pomme-env


In [4]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from time import strftime
from collections import deque, namedtuple
from PIL import Image
import cv2
import IPython.display
from functools import partial
from itertools import chain
import random

import gym
from gym import Env, Wrapper

from pommerman import make
from pommerman.agents import BaseAgent, RandomAgent, SimpleAgent
from graphic_pomme_env import graphic_pomme_env
from graphic_pomme_env.wrappers import PommerEnvWrapperFrameSkip2

print('''Hint: just ignore the error "Import error NSDE! You will not be able to render --> Cannot connect to 'None'"''')
pomenvs = [es.id for es in gym.envs.registry.all() if es.id.startswith('Pomme')]
print("\n".join(pomenvs))
res = graphic_pomme_env.load_resources()

Import error NSDE! You will not be able to render --> Cannot connect to "None"
['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']
/usr/local/lib/python3.7/dist-packages
['/usr/local/lib/python3.7/dist-packages', '', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']
GraphicPommeFFAFast-v0 env is registered.
GraphicOneVsOne-v0 env is registered.
GraphicOVOCompact-v0 env is registered.
GraphicOVONano-v0 env is registered.
Hint: just ignore the error "Import error NSDE! You will not be able to render --> Cannot connect to 'None'"
PommeFFACompetition-v0
PommeFFACompeti

UnidentifiedImageError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/drl_final_challenge/'

In [ ]:
USE_CUDA = torch.cuda.is_available()
Transition = namedtuple("Transition", ["state", "action", "reward", "next_state", "done"])
device = 'cuda'
print(USE_CUDA)

In [ ]:
NUM_ACTIONS = 6

# Replay Buffer

In [ ]:
class ReplayBuffer():
  def __init__(self, num_actions, memory_len = 10000):
      self.memory_len = memory_len
      self.transition = []
      self.num_actions = num_actions

  def add(self, state, action, reward, next_state, done):
      if self.length() > self.memory_len:
        self.remove()
      self.transition.append(Transition(state, action, reward, next_state, done))

  def sample_batch(self, batch_size = 32):
      minibatch = random.sample(self.transition, batch_size)

      states_mb, a_, reward_mb, next_states_mb, done_mb = map(np.array, zip(*minibatch))

      mb_reward = torch.from_numpy(reward_mb).to(device)
      mb_done = torch.from_numpy(done_mb.astype(int)).to(device)

      a_mb = np.zeros((a_.size, self.num_actions))
      a_mb[np.arange(a_.size), a_] = 1
      mb_a = torch.from_numpy(a_mb).to(device)

      return states_mb, mb_a, mb_reward, next_states_mb, mb_done

  def length(self):
      return len(self.transition)

  def remove(self):
      self.transition.pop(0)

# Agent and Opponents

In [ ]:
# Code Block: (b)
class DQNNetwork_atari(nn.Module):

    def __init__(self, num_actions = 6):
        super().__init__()
        # Define your architecture

        self.num_actions = num_actions
        # Network Architecture
        self.features = nn.Sequential(
            nn.Conv2d(5, 32, kernel_size=8, stride=4), nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.ReLU()
          )
          
        self.fc = nn.Sequential(
            nn.Linear(384, 128), nn.ReLU(),
            nn.Linear(128, num_actions)
          )

    def forward(self, x):
      # output forward should always be q values for all actions
        if len(x.size()) == 3:
          x = x.unsqueeze(dim=0)
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
# Update Target network
# Code Block: (h)
def soft_update(local_model, target_model, tau):
    """Soft update model parameters.
    θ_target = τ*θ_local + (1 - τ)*θ_target
    """
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

#### Some useful opponent actor functions

In [ ]:
def idle_actor(frame_stack):
    del frame_stack
    return 0
  
def random_actor(frame_stack):
    del frame_stack
    return np.random.randint(NUM_ACTIONS)

def no_bomb_random_actor(frame_stack):
    del frame_stack
    return np.random.randint(NUM_ACTIONS-1)
  

#### Load Agent and Opponents

In [ ]:
net1 = ConvertModel(onnx.load("model.onnx"), experimental=True)             # score 0.68
net2 = ConvertModel(onnx.load("submission_onnx.onnx"), experimental=True)   # score 0.10
net3 = ConvertModel(onnx.load("submission_final.onnx"), experimental=True)  # score 0.02
net4 = ConvertModel(onnx.load("submission_other.onnx"), experimental=True)  # score 0.56
net5 = ConvertModel(onnx.load("submission_first.onnx"), experimental=True)  # score 0.62


num_actions = 6
#Function from first script to define agent from net
def model_actor(frame_stack, model):
    obs = torch.from_numpy(np.array(frame_stack.get_observation())).to(device)
    net_out = model(obs).detach().cpu().numpy()
    action = np.argmax(net_out)
    return action

/usr/local/lib/python3.7/dist-packages/onnx2pytorch/convert/layer.py:25: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  layer.weight.data = torch.from_numpy(numpy_helper.to_array(weight))
/usr/local/lib/python3.7/dist-packages/onnx2pytorch/convert/model.py:75: UserWarning: Using experimental implementation that allows 'batch_size > 1'.Batchnorm layers could potentially produce false outputs.
  "Using experimental implementation that allows 'batch_size > 1'."


In [ ]:
#get dqn networks with weights of the best actor (net1). I got some convertnet errors otherwise
num_actions = 6
dqn = DQNNetwork_atari(num_actions=num_actions).to(device)
dqn_target = DQNNetwork_atari(num_actions=num_actions).to(device)

#get the learned weights from the best actor
target_state_dict = dqn.state_dict()
for idx,i in enumerate(target_state_dict):
  target_state_dict[i] = net1.state_dict()[list(net1.state_dict().keys())[idx]]
dqn.load_state_dict(target_state_dict)

#get the learned weights from the best actor for the target network
target_state_dict = dqn_target.state_dict()
for idx,i in enumerate(target_state_dict):
  target_state_dict[i] = net1.state_dict()[list(net1.state_dict().keys())[idx]]
dqn_target.load_state_dict(target_state_dict)

<All keys matched successfully>

In [ ]:
net1.train()
net2.train()
net3.train()
net4.train()
net5.train()
model2 = net2.to(device)
model3 = net3.to(device)
model4 = net4.to(device)
model5 = net5.to(device)
opponent1 = partial(model_actor, model=model2) # onnx
opponent2 = partial(model_actor, model=model3) # final
opponent3 = partial(model_actor, model=model4) # other
opponent4 = partial(model_actor, model=model5) # first

actors = [None, opponent1, opponent3, opponent4, no_bomb_random_actor, opponent2] #None is the simple agent from the first challenge
opponent_strings = ['simple agent', 'opponent1', 'opponent3', 'opponent4', 'no_bomb_random_actor', 'opponent2']
num_episodes = [1, 1, 1, 1, 1, 1]
actor_curriculum = chain.from_iterable([[actor] * num_episodes for num_episodes, actor in zip(num_episodes, actors)])

# Training

#### Hyperparameters

In [ ]:
# Hyperparameters
num_episodes_per_run = 2000 # number of episodes
buffer_size = 10 ** 5 * 3 # size of the buffer to use
epsilon = 0.3 # prob of selecting random action a
timesteps = 0 # counter for number of frames
minibatch_size = 64 # size of the minibatch sampled
gamma = 0.99 # discount factor
eval_episode = 100
num_eval = 10
tau = 1e-3 # for updating the target network
learning_rate = 0.00001 
update_after = 2000 # update after num time steps
epsilon_decay = 9**3 
epsilon_ub = 0.3 
epsilon_lb = 0.02

#### Train Loop

Training uses first the simple agent but with a different start position than in the first stage. Then it trains against our other agents.
The number of episodes is actually multiplied with num_episodes_per_run. The start position is alternatingly switched with each num_episodes. So now the first 100 episodes with start position 1, then 100 with start position 0, then the next agent.

In [ ]:
start_pos = 1
episode = 0
count = 0
for actor, opponent_string in list(zip(actor_curriculum, opponent_strings)):
  print()
  print('Opponent: {}'.format(opponent_string))
  start_pos = (1 if start_pos == 0 else 0)
  env = PommerEnvWrapperFrameSkip2(num_stack=5, start_pos=start_pos, opponent_actor = actor, board='GraphicOVOCompact-v0')

  returns = []
  returns_50 = deque(maxlen=50)
  losses = []
  buffer = ReplayBuffer(num_actions=num_actions, memory_len=buffer_size)
  optimizer = optim.Adam(dqn.parameters(), lr=learning_rate)
  mse = torch.nn.MSELoss()

  player_obs, opp_obs = env.reset()

  timesteps = 0
 
  for i in range(num_episodes_per_run):
    ret = 0
    done = False
    while not done:
      # Decay epsilon :
      epsilon = max(epsilon_lb, epsilon_ub - timesteps/ epsilon_decay)
      # action selection
      if np.random.choice([0,1], p=[1-epsilon,epsilon]) == 1:
        a = np.random.randint(low=0, high=num_actions, size=1)[0]
      else:
        player_obs = np.array(player_obs).astype(np.uint8)
        net_out = dqn(torch.from_numpy(player_obs).type(torch.float32).to(device)).detach().cpu().numpy()
        a = np.argmax(net_out)

      player_step, opponent_step = env.step(a)
      player_next_obs, player_r, done, info = player_step
      # opponent_next_obs, opponent_r, done, info = opponent_step

      # next_state, r, done, info = env.step(a)
      ret = ret + player_r
      player_next_obs = np.array(player_next_obs).astype(np.uint8)
      # store transition in the replay buffer 
      # Code Block: (a)
      buffer.add(player_obs, a, player_r, player_next_obs, done)
      player_obs = player_next_obs
      timesteps = timesteps + 1

      # update policy using temporal difference
      if buffer.length() > minibatch_size and buffer.length() > update_after:
        optimizer.zero_grad()

        # Sample a minibatch randomly
        # Code Block: (a)
        states, actions, rewards, next_states, batch_done = buffer.sample_batch(batch_size=minibatch_size)
        
        # Compute q values for states
        # Code Block: (g)
        q_values = dqn(torch.from_numpy(states).type(torch.float32).to(device))

        # compute the targets for training
        # Code Block: (g)
        q_values_target = dqn_target(torch.from_numpy(next_states).type(torch.float32).to(device))
        not_done = batch_done==0
        targets = rewards.clone()
        rewards_not_last_state = rewards + gamma * torch.max(q_values_target, 1).values
        targets[not_done] = rewards_not_last_state.long()[not_done]
        targets = targets.type(torch.FloatTensor).to(device)

        # compute the predictions for training
        # Code Block: (g)
        predictions = q_values[actions==1]

        # Update loss: mse = mean squared error
        loss = mse(predictions, targets) 
        loss.backward(retain_graph=False)
        optimizer.step()
        losses.append(loss.item())
  
        # Update target network
        # Code Block: (h)
        soft_update(dqn, dqn_target, tau)

        # Buffer update
        if prioritized_replay:
          buffer.update_td_errors(indices, torch.abs(predictions - targets).detach().cpu())
      if done:
        player_obs, opp_obs = env.reset()
        # print("Episode: ", i, " ", ret)
        break
    if count % 1000 == 0:
      torch.save(dqn.state_dict(), path + 'checkpoint_dqn_{}.pth'.format(count))
      torch.save(dqn_target.state_dict(), path + 'checkpoint_dqn_target_{}.pth'.format(count))
    count += 1
    returns.append(ret)
    returns_50.append(ret)
    if i % 50 == 0:
      print('\rEpisode {}\tAverage Score: {:.2f}'.format(i, np.mean(returns_50)))  
    episode = episode + 1

    #print('\rStart Position: {}'.format(start_pos) )


Opponent: simple agent
Episode 0	Average Score: -1.00
Episode 50	Average Score: 0.06
Episode 100	Average Score: -0.34
Episode 150	Average Score: -0.58
Episode 200	Average Score: -0.62
Episode 250	Average Score: -0.48
Episode 300	Average Score: -0.52
Episode 350	Average Score: -0.56
Episode 400	Average Score: -0.50
Episode 450	Average Score: -0.58
Episode 500	Average Score: -0.52
Episode 550	Average Score: -0.56
Episode 600	Average Score: -0.66
Episode 650	Average Score: -0.46
Episode 700	Average Score: -0.76
Episode 750	Average Score: -0.44
Episode 800	Average Score: -0.58
Episode 850	Average Score: -0.62
Episode 900	Average Score: -0.44
Episode 950	Average Score: -0.44
Episode 1000	Average Score: -0.78
Episode 1050	Average Score: -0.50
Episode 1100	Average Score: -0.58
Episode 1150	Average Score: -0.50
Episode 1200	Average Score: -0.36
Episode 1250	Average Score: -0.56
Episode 1300	Average Score: -0.74
Episode 1350	Average Score: -0.38
Episode 1400	Average Score: -0.54
Episode 1450	A

# Save agent

In [ ]:
# save model parameters for submitting solution using the following line of code
state = env.reset()
torch.onnx.export(dqn.cpu(), # model
                  torch.tensor(state, dtype=torch.float), # example model input
                  path + "submission.onnx", # name of the submission
                  export_params=True, # save trained parameters
                  opset_version=10,
                  do_constant_folding=True)